# Create DataAccuracy Image for Corrected Fixations Iteration 4
Status: Implemented
 -- Fully automatized

In [1]:
import gc
import json
import warnings

import pandas as pd
from tqdm.notebook import tqdm
from utils.utils import *

warnings.filterwarnings("ignore")

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
sequence_orders = get_sequence_order_per_participant()
assert (set(folders.keys()) == set(sequence_orders.keys()))

In [3]:
global FIXATION_CORRECTION_ITERATION 
FIXATION_CORRECTION_ITERATION[0] = 4
print('Fixation correction iteration:', current_fixation_correction_iteration())
print('Previous Fixation correction iteration:', previous_fixation_correction_iteration())

Fixation correction iteration: 4
Previous Fixation correction iteration: 3


### Take into account previous iterations

In [4]:
if current_fixation_correction_iteration() == 1:
    snippets = {p: sequence_orders[p][SNIPPET].to_list()
                for p in sequence_orders}
else:  # get snippets to rework for this round
    snippets = {p: list(pd.read_csv(get_snippet_rework_paths(
        p), sep=SEPARATOR, index_col=False, )[SNIPPET].to_list()) for p in folders}

In [5]:
manual_evaluation_paths = exist_previous_manual_accuracy_evaluation_paths()
x_offsets = get_x_offsets()
x_offsets

{'003': {},
 '004': {},
 '005': {},
 '007': {},
 '008': {},
 '009': {},
 '010': {},
 '011': {},
 '012': {'v0': 100, 'v1': 50, 'v2': 80},
 '013': {'v0': 20},
 '014': {},
 '015': {},
 '016': {},
 '017': {},
 '018': {},
 '019': {},
 '020': {},
 '021': {},
 '022': {},
 '023': {},
 '024': {},
 '101': {},
 '102': {},
 '106': {}}

### Auto-correction
* generate accuracy images to compare fixation data (original and corrected versions)

In [6]:
# Iterate over all participants
for participant in tqdm(folders):
    print('----------------------------------------------')
    print(PARTICIPANT, participant)
    if not snippets[participant]:
        print('no snippets to (re-)work found.')
        continue

    print('(01/04) Retrieve fixation data')
    participant_snippet_fixations = {}
    not_available = set()
    for snippet in tqdm(snippets[participant]):
        snippet_fixations = {}
        for algo in FIXATION_CORRECTION_ALGORITHMS:
            # select outliers removed version only if for original fixations
            snippet_fixations_path = get_fixations_path(participant, snippet, modified_version=algo,
                                                        no_outliers_if_exists=(algo == FIXATION_CORRECTION_ALGORITHM_ORIGINAL))
            if snippet_fixations_path.exists():
                snippet_fixations[algo] = pd.read_csv(
                    snippet_fixations_path, index_col=False, sep=SEPARATOR, dtype={PARTICIPANT: str})
        participant_snippet_fixations[snippet] = snippet_fixations

    print('(02/04) Add constant offset in x direction if given')
    # calculate participant / version offset into data and create views based on that
    snippets_with_offset = {}
    if participant in x_offsets:
        for snippet in tqdm(participant_snippet_fixations):
            x_offset = get_snippet_x_offset(x_offsets[participant], snippet)
            if x_offset:
                # add offset to participant and corrected fixations
                for fixation_data in participant_snippet_fixations[snippet].values():
                    fixation_data[FIXATION_COLUMN_X] = fixation_data[FIXATION_COLUMN_X]+x_offset
                snippets_with_offset[snippet] = x_offset

    print('(03/04) Generate comparing snippet fixation image for snippet')
    excluded_snippets = {}
    for snippet in tqdm(participant_snippet_fixations):
        if not participant_snippet_fixations[snippet]:
            excluded_snippets[snippet] = f'''01d5_PRE_VIS_Snippet_Correction_Accuracy_Image-it{
                current_fixation_correction_iteration()}: No fixation or auto-correction data available.'''
            continue
        success_snippet = create_accuracy_image(
            participant, snippet, participant_snippet_fixations[snippet], snippets_with_offset[snippet] if snippet in snippets_with_offset else 0)
        if not success_snippet:
            excluded_snippets[snippet] = f'''01d5_PRE_VIS_Snippet_Correction_Accuracy_Image-it{
                current_fixation_correction_iteration()}: Auto-corrected image for snippet fixations could not be created.'''

    print('(04/04) Adapt exclusion file for snippets without fixation accuracy images')
    if excluded_snippets:
        print(excluded_snippets)
        update_exclusions(participant, SNIPPET, VISUAL, excluded_snippets)

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
no snippets to (re-)work found.
----------------------------------------------
Participant 004
no snippets to (re-)work found.
----------------------------------------------
Participant 005
no snippets to (re-)work found.
----------------------------------------------
Participant 007
no snippets to (re-)work found.
----------------------------------------------
Participant 008
no snippets to (re-)work found.
----------------------------------------------
Participant 009
no snippets to (re-)work found.
----------------------------------------------
Participant 010
no snippets to (re-)work found.
----------------------------------------------
Participant 011
no snippets to (re-)work found.
----------------------------------------------
Participant 012
(01/04) Retrieve fixation data


  0%|          | 0/4 [00:00<?, ?it/s]

(02/04) Add constant offset in x direction if given


  0%|          | 0/4 [00:00<?, ?it/s]

(03/04) Generate comparing snippet fixation image for snippet


  0%|          | 0/4 [00:00<?, ?it/s]

(04/04) Adapt exclusion file for snippets without fixation accuracy images
----------------------------------------------
Participant 013
no snippets to (re-)work found.
----------------------------------------------
Participant 014
no snippets to (re-)work found.
----------------------------------------------
Participant 015
no snippets to (re-)work found.
----------------------------------------------
Participant 016
no snippets to (re-)work found.
----------------------------------------------
Participant 017
no snippets to (re-)work found.
----------------------------------------------
Participant 018
no snippets to (re-)work found.
----------------------------------------------
Participant 019
no snippets to (re-)work found.
----------------------------------------------
Participant 020
no snippets to (re-)work found.
----------------------------------------------
Participant 021
no snippets to (re-)work found.
----------------------------------------------
Participant 022
no snip

### Create evaluation form for this round's manual evaluation

In [7]:
# Iterate over all participants
for participant in tqdm(folders):
    print('----------------------------------------------')
    print(PARTICIPANT, participant)
    
    print('(01/02) Retrieve fixation data')
    participant_snippet_fixations = {}
    not_available = set()
    for snippet in tqdm(snippets[participant]):
        snippet_fixations = {}
        for algo in FIXATION_CORRECTION_ALGORITHMS:
            # select outliers removed version only if for original fixations
            snippet_fixations_path = get_fixations_path(participant, snippet, modified_version=algo,
                                                        no_outliers_if_exists=(algo == FIXATION_CORRECTION_ALGORITHM_ORIGINAL))
            if snippet_fixations_path.exists():
                snippet_fixations[algo] = SNIPPET
        participant_snippet_fixations[snippet] = snippet_fixations

    print('(02/02) Prepare data for manual accuracy evaluation file')
    manual_evaluation = create_manual_accuracy_evaluation(
        participant_snippet_fixations)
    if current_fixation_correction_iteration() > 1:
        # add manual evaluation from previous iteration
        previous_manual_evaluation = pd.read_csv(
            manual_evaluation_paths[participant], sep=SEPARATOR, index_col=False, dtype={PARTICIPANT: str})
        new_manual_evaluation = manual_evaluation
        manual_evaluation = pd.concat(
            [previous_manual_evaluation, new_manual_evaluation]).drop_duplicates([SNIPPET], keep='last')
    manual_evaluation.to_csv(get_manual_accuracy_evaluation_path(
        participant, performed=not snippets[participant]), sep=SEPARATOR, index=False)

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
(01/02) Retrieve fixation data


0it [00:00, ?it/s]

(02/02) Prepare data for manual accuracy evaluation file
----------------------------------------------
Participant 004
(01/02) Retrieve fixation data


0it [00:00, ?it/s]

...

(02/02) Prepare data for manual accuracy evaluation file
----------------------------------------------
Participant 102
(01/02) Retrieve fixation data


0it [00:00, ?it/s]

(02/02) Prepare data for manual accuracy evaluation file
----------------------------------------------
Participant 106
(01/02) Retrieve fixation data


0it [00:00, ?it/s]

(02/02) Prepare data for manual accuracy evaluation file
